In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

# Pandas Group Operations

Let's next go over grouped operations with pandas. This section of the pandas library does not have as much feature bloat as other parts, which is nice. And the community is starting to narrow around a couple of operations that are core to grouped operations. We'll be going over these operations with particular emphasis on groupby and agg:

* groupby
* agg
* filter
* transform

Check out the full documentation [here](http://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html), but be warned it is a bit long :)

Let's start with our good old tips dataset:

In [2]:
tips = sns.load_dataset('tips', cache=False)
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


### Groupby

A grouped operation starts by specifying which groups of data that we would want to operate over. There are many ways of making groups, but the tool that pandas uses to make groups of data, is `groupby`

In [4]:
tips_gb = tips.groupby(['sex', 'smoker'])
tips_gb

Groupby works by telling pandas a couple of columns. Pandas will look in your data and see every unique combination of the columns that you specify. Each unique combination is a group. So in this case we will have four groups: male smoker, female smoker, male non-smoker, female non-smoker.

The groupby object by itself is not super important.

Once we have these groups (specified in the groupby object), we can do three types of operations on it (with the most important being agg)

### Agg

The aggregate operation aggregates all the data in these groups into one value. You use a dictionary to specify which values you'd like. For example look below, we are asking for both the mean and the min value of the tip column for each group:

In [5]:
tips_agg = tips_gb.agg({
    'tip':['mean', 'min'], 
    'total_bill':'count'
})

tips_agg

tip       total_bill
                   mean   min      count
sex    smoker                           
Male   Yes     3.051167  1.00         60
       No      3.113402  1.25         97
Female Yes     2.931515  1.00         33
       No      2.773519  1.00         54

So notice that we get both a multi-index for both the index and the columns. We can always get rid of the multi-index with a `reset_index`

In [8]:
tips_agg['tip']['mean']['Male']['Yes']

3.0511666666666666

In [9]:
tips_agg.loc[('Male', 'Yes'), ('tip', 'mean')]

3.0511666666666666

In [10]:
tips_agg.reset_index()

sex smoker       tip       total_bill
                      mean   min      count
0    Male    Yes  3.051167  1.00         60
1    Male     No  3.113402  1.25         97
2  Female    Yes  2.931515  1.00         33
3  Female     No  2.773519  1.00         54

In [12]:
# Important Trick to get rid of multilevel columns
tips_agg.columns.values

array([('tip', 'mean'), ('tip', 'min'), ('total_bill', 'count')],
      dtype=object)

In [14]:
my_new_cols = ['__'.join(col).strip() for col in tips_agg.columns.values]    # HW: How to do this via stack/unstack, and pivot/melt
print(my_new_cols)
tips_agg.columns = my_new_cols

['tip__mean', 'tip__min', 'total_bill__count']


In [15]:
tips_agg

tip__mean  tip__min  total_bill__count
sex    smoker                                        
Male   Yes      3.051167      1.00                 60
       No       3.113402      1.25                 97
Female Yes      2.931515      1.00                 33
       No       2.773519      1.00                 54

That is about it for the aggregation, you can find some common aggregation functions listed [here](http://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#aggregation)

### Filter

The next common group operation is a filter. This one is pretty simple, we filter out member of groups that don't meet our criteria.

For example let's only look at the least busy times the place is open. One way we might do that is exclude all times above the median from the analysis

In [16]:
tips_gb = tips.groupby(['day', 'time'])

In [ ]:
tips_gb.agg({'size': 'sum'})

In [22]:
grouped_median_size = tips_gb.agg({'size': 'sum'}).median()[0]

In [23]:
tips_gb.filter(lambda group: group['size'].sum() < grouped_median_size)

,total_bill,tip,sex,smoker,day,time,size
220,12.16,2.20,Male,Yes,Fri,Lunch,2
221,13.42,3.48,Female,Yes,Fri,Lunch,2
222,8.58,1.92,Male,Yes,Fri,Lunch,1
223,15.98,3.00,Female,No,Fri,Lunch,3
224,13.42,1.58,Male,Yes,Fri,Lunch,2
225,16.27,2.50,Female,Yes,Fri,Lunch,2
226,10.09,2.00,Female,Yes,Fri,Lunch,2
243,18.78,3.00,Female,No,Thur,Dinner,2


That's honestly about it. I don't use this functionality too much, but it's pretty simple and I don't think it complicates things too much, so may as well throw it in.

### Transform

The final group operation is transform. This uses group information to apply transformations to individual data points. For example look below: each day let's divide by the bill and tip by the average amount spent on that day. That way we can look at how much that bill differs from the average of that day

In [24]:
tips_gb = tips.groupby(['day'])

In [26]:
tips_gb[['total_bill', 'tip']].transform(lambda col: col / col.mean())

,total_bill,tip
0,0.793554,0.310279
1,0.482952,0.509964
2,0.981317,1.075225
3,1.106025,1.016856
4,1.148529,1.109018
...,...,...
239,1.420159,1.977880
240,1.329656,0.668203
241,1.109025,0.668203
242,0.871761,0.584677


I think I have only ever used this function for normalization, but it is pretty straight forwards and intuitive, so I'm fine with the added flexibility.

## Stacking

<div align = "center">  
<img src = "https://miro.medium.com/v2/resize:fit:720/format:webp/0*2VhoZlZC9njz1txT.png" width = "700" />
</div>

In pandas terminology stacking is referred as transforming horizontal column names to vertical column values . stack function converts the data from a wide to long format.Below mentioned are the input parameters to the function.

level: Prescribed level(s) to stack from column axis onto index axis (int,str, list, default -1).
dropna: Whether to drop rows with missing values in the resulting frame, defaults (bool, default True).
Let us start by anaylsing the sample dataset shown below.

In [27]:
import numpy as np
import pandas as pd

In [28]:
df_sensors = pd.DataFrame({
    'Sensor':['sensor1', 'sensor1', 'sensor1', 'sensor2', 'sensor2', 'sensor2'],
    'Metric':['Pressure', 'Temperature', 'Flow', 'Pressure', 'Temperature', 'Flow'],
    '2017':[79,59,46,62,57,87],
    '2018':[80,67,83,85,41,97]
})
df_sensors

,Sensor,Metric,2017,2018
0,sensor1,Pressure,79,80
1,sensor1,Temperature,59,67
2,sensor1,Flow,46,83
3,sensor2,Pressure,62,85
4,sensor2,Temperature,57,41
5,sensor2,Flow,87,97


We can see that the two year values 2017 and 2018 are present as column names, but ideally both of them depict a single variable which can named as Year. We can resturcture the two Year values as a single column using stack as shown below.

In [29]:
# Stacking the Year columns into a single column.
(df_sensors.set_index(['Sensor', 'Metric']).stack().reset_index().rename(columns={'level_2': 'Year', 0: 'Value'}))

,Sensor,Metric,Year,Value
0,sensor1,Pressure,2017,79
1,sensor1,Pressure,2018,80
2,sensor1,Temperature,2017,59
3,sensor1,Temperature,2018,67
4,sensor1,Flow,2017,46
5,sensor1,Flow,2018,83
6,sensor2,Pressure,2017,62
7,sensor2,Pressure,2018,85
8,sensor2,Temperature,2017,57
9,sensor2,Temperature,2018,41


In [32]:
df_sensors.stack()

0  Sensor        sensor1
   Metric       Pressure
   2017               79
   2018               80
1  Sensor        sensor1
   Metric    Temperature
   2017               59
   2018               67
2  Sensor        sensor1
   Metric           Flow
   2017               46
   2018               83
3  Sensor        sensor2
   Metric       Pressure
   2017               62
   2018               85
4  Sensor        sensor2
   Metric    Temperature
   2017               57
   2018               41
5  Sensor        sensor2
   Metric           Flow
   2017               87
   2018               97
dtype: object

In [33]:
df_sensors.set_index(['Metric'])

,Sensor,2017,2018
Metric,,,
Pressure,sensor1,79,80
Temperature,sensor1,59,67
Flow,sensor1,46,83
Pressure,sensor2,62,85
Temperature,sensor2,57,41
Flow,sensor2,87,97


In [34]:
df_sensors.set_index(['Metric']).stack()

Metric             
Pressure     Sensor    sensor1
             2017           79
             2018           80
Temperature  Sensor    sensor1
             2017           59
             2018           67
Flow         Sensor    sensor1
             2017           46
             2018           83
Pressure     Sensor    sensor2
             2017           62
             2018           85
Temperature  Sensor    sensor2
             2017           57
             2018           41
Flow         Sensor    sensor2
             2017           87
             2018           97
dtype: object

In [35]:
df_sensors.set_index(['Metric']).stack().reset_index()  #Stacked output

,Metric,level_1,0
0,Pressure,Sensor,sensor1
1,Pressure,2017,79
2,Pressure,2018,80
3,Temperature,Sensor,sensor1
4,Temperature,2017,59
5,Temperature,2018,67
6,Flow,Sensor,sensor1
7,Flow,2017,46
8,Flow,2018,83
9,Pressure,Sensor,sensor2


In [36]:
# Stacking the Year Columns
df_sensors.set_index(['Sensor', 'Metric']).stack()

Sensor   Metric           
sensor1  Pressure     2017    79
                      2018    80
         Temperature  2017    59
                      2018    67
         Flow         2017    46
                      2018    83
sensor2  Pressure     2017    62
                      2018    85
         Temperature  2017    57
                      2018    41
         Flow         2017    87
                      2018    97
dtype: int64

In [37]:
# Stacking the Year Columns
df_sensors.set_index(['Sensor', 'Metric']).stack().reset_index()

,Sensor,Metric,level_2,0
0,sensor1,Pressure,2017,79
1,sensor1,Pressure,2018,80
2,sensor1,Temperature,2017,59
3,sensor1,Temperature,2018,67
4,sensor1,Flow,2017,46
5,sensor1,Flow,2018,83
6,sensor2,Pressure,2017,62
7,sensor2,Pressure,2018,85
8,sensor2,Temperature,2017,57
9,sensor2,Temperature,2018,41


In [38]:
df_sensors.set_index(['Sensor', 'Metric']).stack().reset_index().rename(columns={'level_2': 'Year', 0: 'Value'})

,Sensor,Metric,Year,Value
0,sensor1,Pressure,2017,79
1,sensor1,Pressure,2018,80
2,sensor1,Temperature,2017,59
3,sensor1,Temperature,2018,67
4,sensor1,Flow,2017,46
5,sensor1,Flow,2018,83
6,sensor2,Pressure,2017,62
7,sensor2,Pressure,2018,85
8,sensor2,Temperature,2017,57
9,sensor2,Temperature,2018,41


In [63]:
# Add a column level Value on top of year
df_sensor2 = df_sensors.set_index(['Sensor', 'Metric'])
df_sensor2.stack().reset_index()

,Sensor,Metric,level_2,0
0,sensor1,Pressure,2017,79
1,sensor1,Pressure,2018,80
2,sensor1,Temperature,2017,59
3,sensor1,Temperature,2018,67
4,sensor1,Flow,2017,46
5,sensor1,Flow,2018,83
6,sensor2,Pressure,2017,62
7,sensor2,Pressure,2018,85
8,sensor2,Temperature,2017,57
9,sensor2,Temperature,2018,41


In [53]:
df_sensor2.columns

Index(['2017', '2018'], dtype='object')

In [54]:
# new_col_names = ['recorded_values_' + col for col in df_sensor2.columns]
# df_sensor2.columns = new_col_names
# df_sensor2

In [55]:
df_sensor2.columns = pd.MultiIndex.from_product([['Value'], df_sensor2.columns])
df_sensor2

Value     
                     2017 2018
Sensor  Metric                
sensor1 Pressure       79   80
        Temperature    59   67
        Flow           46   83
sensor2 Pressure       62   85
        Temperature    57   41
        Flow           87   97

In [58]:
df_sensor2.stack()

Value
Sensor  Metric                 
sensor1 Pressure    2017     79
                    2018     80
        Temperature 2017     59
                    2018     67
        Flow        2017     46
                    2018     83
sensor2 Pressure    2017     62
                    2018     85
        Temperature 2017     57
                    2018     41
        Flow        2017     87
                    2018     97

In [43]:
df_sensor2.columns

MultiIndex([('Value', '2017'),
            ('Value', '2018')],
           )

Points to Remember:

By default stack takes in all columns present at the inner most level in the column axis and stacks them row wise . If you want any column to stay in index you need to set your index column explicitly using `pd.DataFrame.set_index` method before performing stack and then use `pd.DataFrame.reset_index()` to convert the output to a data frame.

# Unstacking

<div align = "center">  
<img src = "https://miro.medium.com/v2/resize:fit:720/format:webp/0*AsK-EkbNYMGb6WtN.png" width = "600" />
</div>

unstack is an inverse operation of stack i.e it converts a data frame from long to wide format. By default it takes the inner most index values and returns a data frame by reshaping them as the columns. Below are the parameters for the unstack method.

- level: level(s) of index to unstack, defaults to -1 (int or string or list of these)
- fill_value: replace NAN with value specified if unstack produces missing values.
- In order to understand the unstack method let us take the stacked output and see how we can invert the stack operation.

In [68]:
df_sensors

,Sensor,Metric,2017,2018
0,sensor1,Pressure,79,80
1,sensor1,Temperature,59,67
2,sensor1,Flow,46,83
3,sensor2,Pressure,62,85
4,sensor2,Temperature,57,41
5,sensor2,Flow,87,97


In [64]:
# Loadind stacked data into a dataframe
df_stacked = (df_sensors.set_index(['Sensor', 'Metric'])
              .stack()
              .reset_index()
              .rename(columns={'level_2': 'Year', 0: 'Value'}))
df_stacked

,Sensor,Metric,Year,Value
0,sensor1,Pressure,2017,79
1,sensor1,Pressure,2018,80
2,sensor1,Temperature,2017,59
3,sensor1,Temperature,2018,67
4,sensor1,Flow,2017,46
5,sensor1,Flow,2018,83
6,sensor2,Pressure,2017,62
7,sensor2,Pressure,2018,85
8,sensor2,Temperature,2017,57
9,sensor2,Temperature,2018,41


We can now invert the stacked output by following the below three steps:

- Set the columns [Sensor, Metric and Year] as index.
- Apply unstack.
- Process the output to a consumable form.

In [66]:
# Setting the index columns
df_stacked.set_index(['Sensor', 'Metric', 'Year'])

Value
Sensor  Metric      Year       
sensor1 Pressure    2017     79
                    2018     80
        Temperature 2017     59
                    2018     67
        Flow        2017     46
                    2018     83
sensor2 Pressure    2017     62
                    2018     85
        Temperature 2017     57
                    2018     41
        Flow        2017     87
                    2018     97

In [67]:
df_stacked.set_index(['Sensor', 'Metric', 'Year']).unstack()

Value     
Year                 2017 2018
Sensor  Metric                
sensor1 Flow           46   83
        Pressure       79   80
        Temperature    59   67
sensor2 Flow           87   97
        Pressure       62   85
        Temperature    57   41

In [71]:
dfx = df_stacked.set_index(['Sensor', 'Metric', 'Year']).unstack().reset_index()
dfx

Sensor       Metric Value     
Year                        2017 2018
0     sensor1         Flow    46   83
1     sensor1     Pressure    79   80
2     sensor1  Temperature    59   67
3     sensor2         Flow    87   97
4     sensor2     Pressure    62   85
5     sensor2  Temperature    57   41

In [72]:
dfx.columns = dfx.columns.droplevel([0])

In [73]:
dfx          # HW: Rename 2nd and 3rd column

Year,,,2017,2018
0,sensor1,Flow,46,83
1,sensor1,Pressure,79,80
2,sensor1,Temperature,59,67
3,sensor2,Flow,87,97
4,sensor2,Pressure,62,85
5,sensor2,Temperature,57,41


In [75]:
dfx.columns = ['Sensor', 'Metric', '2017', '2018']
dfx

,Sensor,Metric,2017,2018
0,sensor1,Flow,46,83
1,sensor1,Pressure,79,80
2,sensor1,Temperature,59,67
3,sensor2,Flow,87,97
4,sensor2,Pressure,62,85
5,sensor2,Temperature,57,41


Let us see some more examples of unstack for better understanding:

1. Unstacking sensors metric values as columns : We can use unstack operation on the stacked data frame to obtain the desired result as shown below.

In [76]:
df_stacked

,Sensor,Metric,Year,Value
0,sensor1,Pressure,2017,79
1,sensor1,Pressure,2018,80
2,sensor1,Temperature,2017,59
3,sensor1,Temperature,2018,67
4,sensor1,Flow,2017,46
5,sensor1,Flow,2018,83
6,sensor2,Pressure,2017,62
7,sensor2,Pressure,2018,85
8,sensor2,Temperature,2017,57
9,sensor2,Temperature,2018,41


In [77]:
df_stacked.set_index(['Sensor', 'Year', 'Metric'])

Value
Sensor  Year Metric            
sensor1 2017 Pressure        79
        2018 Pressure        80
        2017 Temperature     59
        2018 Temperature     67
        2017 Flow            46
        2018 Flow            83
sensor2 2017 Pressure        62
        2018 Pressure        85
        2017 Temperature     57
        2018 Temperature     41
        2017 Flow            87
        2018 Flow            97

In [78]:
df_stacked.set_index(['Sensor', 'Year', 'Metric']).unstack()

Value                     
Metric        Flow Pressure Temperature
Sensor  Year                           
sensor1 2017    46       79          59
        2018    83       80          67
sensor2 2017    87       62          57
        2018    97       85          41

In [79]:
dfy = df_stacked.set_index(['Sensor', 'Year', 'Metric']).unstack()
dfy.columns = dfy.columns.droplevel([0])
dfy

Metric        Flow  Pressure  Temperature
Sensor  Year                             
sensor1 2017    46        79           59
        2018    83        80           67
sensor2 2017    87        62           57
        2018    97        85           41

2. Unstacking both Year and Metrics as columns : Using the level parameter we can unstack methods to reshape multistack function converts a data frame from wide to long format,ple column values as individual hierarchical columns as shown below.

In [80]:
df_stacked

,Sensor,Metric,Year,Value
0,sensor1,Pressure,2017,79
1,sensor1,Pressure,2018,80
2,sensor1,Temperature,2017,59
3,sensor1,Temperature,2018,67
4,sensor1,Flow,2017,46
5,sensor1,Flow,2018,83
6,sensor2,Pressure,2017,62
7,sensor2,Pressure,2018,85
8,sensor2,Temperature,2017,57
9,sensor2,Temperature,2018,41


In [81]:
df_stacked.set_index(['Sensor', 'Year', 'Metric'])

Value
Sensor  Year Metric            
sensor1 2017 Pressure        79
        2018 Pressure        80
        2017 Temperature     59
        2018 Temperature     67
        2017 Flow            46
        2018 Flow            83
sensor2 2017 Pressure        62
        2018 Pressure        85
        2017 Temperature     57
        2018 Temperature     41
        2017 Flow            87
        2018 Flow            97

In [82]:
df_stacked.set_index(['Sensor', 'Year', 'Metric']).unstack(level=[-2, -1])

Value                                           
Year        2017     2018        2017        2018 2017 2018
Metric  Pressure Pressure Temperature Temperature Flow Flow
Sensor                                                     
sensor1       79       80          59          67   46   83
sensor2       62       85          57          41   87   97

## Melting

<div align = "center">  
<img src = "https://miro.medium.com/v2/resize:fit:720/format:webp/0*O3SGf68-vlON_H8L.png" width = "600" />
</div>

Pandas has different ways to accomplish the same task, the difference being readability and performance. Pandas data frame method name melt works similarly to stack but gives more flexibility.The method takes in the below 5 parameters out of which two parameters namely id_vars and value_vars are crucial to understand how to reshape your data.

- id_vars: list of column names that you want to preserve as columns and not reshape (list,tuple or ndarray)(optional).
- value_vars: list of column names that you want to reshape as columns (list,tuple or ndarray) (optional).
- var_name: Name to use for the variable column, defaults to variable.(scalar)(optional)
- value_name:Name to use for the value column, defaults to `value`(scalar)(optional).
- col_level:If column are multi index then use this level to melt (int or string)(optional).
All of the above parameters mentioned are optional. Let us try to understand the usage of each of the parameters by applying it to our sample dataset.

Working of Melt:

The id_vars or the identification variables remain in the same column but repeat for each of the value_vars
One crucial aspect of melt is that it ignores the values in index, in fact it drops the existing index and replaces it with the RangeIndex. so if you have values in index that you want to keep, you need to do a reset_index before applying melt.
Let’s apply the melt function on our sample sensors data frame.

In [83]:
df_sensors

,Sensor,Metric,2017,2018
0,sensor1,Pressure,79,80
1,sensor1,Temperature,59,67
2,sensor1,Flow,46,83
3,sensor2,Pressure,62,85
4,sensor2,Temperature,57,41
5,sensor2,Flow,87,97


In [84]:
df_sensors.melt(id_vars=['Sensor', 'Metric'])

,Sensor,Metric,variable,value
0,sensor1,Pressure,2017,79
1,sensor1,Temperature,2017,59
2,sensor1,Flow,2017,46
3,sensor2,Pressure,2017,62
4,sensor2,Temperature,2017,57
5,sensor2,Flow,2017,87
6,sensor1,Pressure,2018,80
7,sensor1,Temperature,2018,67
8,sensor1,Flow,2018,83
9,sensor2,Pressure,2018,85


`melt` works similar to `stack` but gives more flexibility. By default melt assigns variable and value as the names for value_vars, this can be overrided by passing the desired column names as an input to the parameters var_name and value_name as shown below.

In [86]:
df_sensors.melt(id_vars=['Sensor', 'Metric'], var_name='Year', value_name='recordings')

,Sensor,Metric,Year,recordings
0,sensor1,Pressure,2017,79
1,sensor1,Temperature,2017,59
2,sensor1,Flow,2017,46
3,sensor2,Pressure,2017,62
4,sensor2,Temperature,2017,57
5,sensor2,Flow,2017,87
6,sensor1,Pressure,2018,80
7,sensor1,Temperature,2018,67
8,sensor1,Flow,2018,83
9,sensor2,Pressure,2018,85


Point to Remember: The advantage of melt over stack is that you can mention the column name which you want to preserve in the index without explicitly setting them as index .

## Pivot

<div align = "center">  
<img src = "https://miro.medium.com/v2/resize:fit:720/format:webp/0*DBIsbGKzrfZSXvPq.png" width = "600" />
</div>

pivot method is similar to unstack but is more easy to use. It takes in 3 parameters (mentioned below) as input which are index, columns and values

- index: Column to use for the new frames index, if none use current index(string or object).
- columns: Column to use for the new frames columns (string or object).
- values : Column(s) to use for the new frames values (string or object).

Working of pivot:
Each parameter takes in a single column as a string.
The index remains in the vertical and becomes the new index.
The values of the columns referenced by columns becomes new column names.
The values referenced by the values are tiled to correspond the intersection of their former index and columns label.
Let us consider the previous melted output and see how to apply pivot in order to reshape it to original form.

In [87]:
# loading melted data as a dataframe
df_melted = df_sensors.melt(id_vars=['Sensor', 'Metric'], var_name='Year', value_name='value')
df_melted

,Sensor,Metric,Year,value
0,sensor1,Pressure,2017,79
1,sensor1,Temperature,2017,59
2,sensor1,Flow,2017,46
3,sensor2,Pressure,2017,62
4,sensor2,Temperature,2017,57
5,sensor2,Flow,2017,87
6,sensor1,Pressure,2018,80
7,sensor1,Temperature,2018,67
8,sensor1,Flow,2018,83
9,sensor2,Pressure,2018,85


In [88]:
df_melted.pivot(index=['Sensor', 'Metric'], columns='Year', values='value')

Year                 2017  2018
Sensor  Metric                 
sensor1 Flow           46    83
        Pressure       79    80
        Temperature    59    67
sensor2 Flow           87    97
        Pressure       62    85
        Temperature    57    41

In [92]:
df_melted.pivot(index=['Sensor', 'Metric'], columns='Year', values='value').reset_index()

Year,Sensor,Metric,2017,2018
0,sensor1,Flow,46,83
1,sensor1,Pressure,79,80
2,sensor1,Temperature,59,67
3,sensor2,Flow,87,97
4,sensor2,Pressure,62,85
5,sensor2,Temperature,57,41


In [91]:
df_melted.set_index(['Sensor', 'Metric']).pivot(columns='Year')['value']  # Never do this, only for understanding purpose

Year                 2017  2018
Sensor  Metric                 
sensor1 Flow           46    83
        Pressure       79    80
        Temperature    59    67
sensor2 Flow           87    97
        Pressure       62    85
        Temperature    57    41

pivot cannot take more than one index at a time.In order to achieve that you will have to explicitly set the index and apply pivot on columns.

## pivot_table

pivot_table is a versatile and flexible function. Below mentioned are the list of the input parameters to the fucntion.. Below mentioned are the input parameters to the function.

- index: column(s) which is intended to stay as index. (column, list, array, Grouper)
- columns: column(s) which are pivoted (column, list, array, Grouper).
- values : column to aggregate.
- aggfunc: function, list of aggregation functions.(defaults to mean)
- fill_value: Scalar to fill for missing values in the result (scalar, default None).
- margins: whether to add all rows/columns (Bool, default False )(eg: subtotal or grand total)
- dropna: Do not include columns whose values for all rows are NaN (Bool, default True).
- margins_name: Name of the row / column that will contain the totals when margins is True (string, default All).

The parameters have changed a bit... please look at the latest pandas documenatation for reference.

Working of pivot_table:
pivot_table is a generalization of pivot that can handle duplicate values for one pivoted index/column pair. Specifically, you can give pivot_table a list of aggregation functions using keyword argument aggfunc. The default aggfunc of pivot_table is numpy.mean.
pivot_table also supports using multiple columns for the index and column of the pivoted table. A hierarchical index will be automatically generated for you.
pivot_table also has some additional default parameters named margins , margins_name which indicates whether to add rows and columns total in the output or not.
Let us see how can we apply pivot_table to the previous melted output in order to convert it to original form.

In [93]:
# loading melted data as a dataframe
df_melted = df_sensors.melt(id_vars=['Sensor', 'Metric'],
                value_vars=['2017', '2018'],
                var_name='Year',
                value_name='value')
df_melted

,Sensor,Metric,Year,value
0,sensor1,Pressure,2017,79
1,sensor1,Temperature,2017,59
2,sensor1,Flow,2017,46
3,sensor2,Pressure,2017,62
4,sensor2,Temperature,2017,57
5,sensor2,Flow,2017,87
6,sensor1,Pressure,2018,80
7,sensor1,Temperature,2018,67
8,sensor1,Flow,2018,83
9,sensor2,Pressure,2018,85


In [94]:
df_melted.pivot_table(index=['Sensor', 'Metric'], columns='Year', values='value')

Year                 2017  2018
Sensor  Metric                 
sensor1 Flow           46    83
        Pressure       79    80
        Temperature    59    67
sensor2 Flow           87    97
        Pressure       62    85
        Temperature    57    41

pivot_table is widely used to summarize the numerical data. You can perform multiple aggregations like count, sort, average, total etc using pivot_table . We can use the parameters aggfunc, margins to perform any no of aggregations at a time on the chosen column values.



In [95]:
# Performing aggregations using aggfunc.
df_melted.pivot_table(index=['Sensor', 'Year'],
                     columns='Metric',
                     values='value',
                     aggfunc=[np.sum, np.mean],
                     fill_value=0,
                     margins=True,
                     margins_name='Total',
                     sort=True)

sum                              mean                       \
Metric       Flow Pressure Temperature Total   Flow Pressure Temperature   
Sensor  Year                                                               
sensor1 2017   46       79          59   184  46.00     79.0          59   
        2018   83       80          67   230  83.00     80.0          67   
sensor2 2017   87       62          57   206  87.00     62.0          57   
        2018   97       85          41   223  97.00     85.0          41   
Total         313      306         224   843  78.25     76.5          56   

                         
Metric            Total  
Sensor  Year             
sensor1 2017  61.333333  
        2018  76.666667  
sensor2 2017  68.666667  
        2018  74.333333  
Total         70.250000

Key Differences between pivot and pivot_table:
- pivot is used for pivoting without aggregation. Therefore, it can’t deal with duplicate values for one index/column pair whereas pivot_table can be used for both the purposes.